# Quickstart: Political Democracy
The model "Poltical Democracy" from Kenneth A. Bollen book is a part of **semopy** package. The model's description and the respective data can be retrieved via invoking *get_model* and *get_data* methods from the *political_democracy* submodule of the package.

In [1]:
import numpy as np
import semopy
from semopy.political_democracy import get_data, get_model

np.random.seed(2019)
model_desc = get_model()
data = get_data()

In [2]:
print(model_desc)

# measurement model
ind60 =~ x1 + x2 + x3
dem60 =~ y1 + y2 + y3 + y4
dem65 =~ y5 + y6 + y7 + y8
# regressions
dem60 ~ ind60
dem65 ~ ind60 + dem60
# residual correlations
y1 ~~ y5
y2 ~~ y4 + y6
y3 ~~ y7
y4 ~~ y8
y6 ~~ y8



Now, we can create an instant of *Model* from *model_desc* and load data into it.

In [3]:
from semopy import Model

model = Model(model_desc)
model.load_dataset(data)

Now we can create an *Optimizer* by passing it *model*. In fact, we can create as many instances of *Optimizer* as we want. Each of them can perform an independent optimization sequence.
Let's say that we want to see estimates provided by minimising Wishart Maximumul Likelihood Ratio, ULS and GLS:

In [4]:
from semopy import Optimizer

opt_mlw = Optimizer(model)
opt_uls = Optimizer(model)
opt_gls = Optimizer(model)

# And now, we run the optimisation sequences.
lf_mlw = opt_mlw.optimize(objective='MLW') # Although MLW is default, we still provide it here for clarity.
lf_uls = opt_uls.optimize(objective='ULS')
lf_gls = opt_gls.optimize(objective='GLS')

print('Resultant objective functions'' values are:')
print('MLW: {:.3f}, ULS: {:.3f}, GLS: {:.3f}'.format(lf_mlw, lf_uls, lf_gls))

Resultant objective functions values are:
MLW: 0.508, ULS: 7.097, GLS: 0.972


Let's also try minimisng the MLW objective but instead of using default SLSQP nonlinear solver we will try using Adam with chunk_size=25 and num_epochs=1000:

In [5]:
opt_mlw_adam = Optimizer(model)
lf_mlw_adam = opt_mlw_adam.optimize(objective='MLW', method='Adam', chunk_size=25, num_epochs=1000)
print('MLW after Adam: {:.3f}'.format(lf_mlw_adam))

MLW after Adam: 0.513


Take a notice that one can't compare results for the same model based on the value of different loss functions. Fit indices (we will compute then in the end of this notebook) are a valid measure.

Also, it can be seen from the code above that we can in fact run another optimisation sequences for the same *Optimizer*, using previous parameters' estimates as starting values:

In [6]:
lf_mlw_adam_slsqp = opt_mlw_adam.optimize(method='SLSQP')
print('MLW after Adam after SLSQP: {:.3f}'.format(lf_mlw_adam_slsqp))

MLW after Adam after SLSQP: 0.508


The *inspector* module of **semopy** contains *inspect* method that is used to retrieve information on parameters' estimates in a user-friendly manner. It has two modes of display - 'list' (the default one) and 'mx'. Let's try the 'list' first:

In [7]:
from semopy.inspector import inspect

print(inspect(opt_mlw, mode='list'))

     lval  op   rval     Value        SE    Z-score       P-value
3      x2  =~  ind60  2.180344  0.138503  15.742231  0.000000e+00
4      x3  =~  ind60  1.818498  0.151953  11.967497  0.000000e+00
5      y2  =~  dem60  1.256759  0.182449   6.888264  5.647705e-12
6      y3  =~  dem60  1.057701  0.151394   6.986417  2.819966e-12
7      y4  =~  dem60  1.264819  0.145013   8.722092  0.000000e+00
8      y6  =~  dem65  1.185704  0.168814   7.023728  2.160272e-12
9      y7  =~  dem65  1.279501  0.159904   8.001694  1.332268e-15
10     y8  =~  dem65  1.265968  0.158112   8.006805  1.110223e-15
0   dem60   ~  ind60  1.482939  0.399133   3.715402  2.028809e-04
1   dem65   ~  dem60  0.837351  0.098353   8.513720  0.000000e+00
2   dem65   ~  ind60  0.572345  0.221309   2.586173  9.704827e-03
11  dem60  ~~  dem60  3.955856  0.921163   4.294415  1.751547e-05
12  dem65  ~~  dem65  0.172495  0.214804   0.803034  4.219551e-01
13  ind60  ~~  ind60  0.448458  0.086695   5.172836  2.305675e-07
14     x1 

We might also want to take a peek at starting values:

In [8]:
print(inspect(opt_mlw, mode='list', what='start'))

     lval  op   rval     Value         SE   Z-score   P-value
3      x2  =~  ind60  1.843737   2.241495  0.822548  0.410765
4      x3  =~  ind60  1.532953   1.802184  0.850609  0.394987
5      y2  =~  dem60  0.908818  17.247656  0.052692  0.957977
6      y3  =~  dem60  0.848845  12.967861  0.065458  0.947810
7      y4  =~  dem60  0.885162  16.123320  0.054899  0.956219
8      y6  =~  dem65  0.729128  14.418897  0.050568  0.959670
9      y7  =~  dem65  0.852861  14.045512  0.060721  0.951581
10     y8  =~  dem65  0.782249  14.947613  0.052333  0.958264
0   dem60   ~  ind60  0.000000   1.134786  0.000000  1.000000
1   dem65   ~  dem60  0.000000   4.275070  0.000000  1.000000
2   dem65   ~  ind60  0.000000   1.162828  0.000000  1.000000
11  dem60  ~~  dem60  0.050000   0.958579  0.052161  0.958401
12  dem65  ~~  dem65  0.050000   1.004401  0.049781  0.960297
13  ind60  ~~  ind60  0.050000   0.072395  0.690654  0.489783
14     x1  ~~     x1  0.264993   0.079626  3.327970  0.000875
15     x

The other mode of display is 'mx'. That's it, matrices with parameters values mapped to their positions will be printed:

In [9]:
print(inspect(opt_mlw, mode='mx'))

Beta:
          dem60  dem65     ind60
dem60  0.000000    0.0  1.482939
dem65  0.837351    0.0  0.572345
ind60  0.000000    0.0  0.000000
Lambda:
       dem60     dem65     ind60
x1  0.000000  0.000000  1.000000
x2  0.000000  0.000000  2.180344
x3  0.000000  0.000000  1.818498
y1  1.000000  0.000000  0.000000
y2  1.256759  0.000000  0.000000
y3  1.057701  0.000000  0.000000
y4  1.264819  0.000000  0.000000
y5  0.000000  1.000000  0.000000
y6  0.000000  1.185704  0.000000
y7  0.000000  1.279501  0.000000
y8  0.000000  1.265968  0.000000
Psi:
          dem60     dem65     ind60
dem60  3.955856  0.000000  0.000000
dem65  0.000000  0.172495  0.000000
ind60  0.000000  0.000000  0.448458
Theta:
          x1        x2      x3        y1        y2        y3        y4  \
x1  0.081547  0.000000  0.0000  0.000000  0.000000  0.000000  0.000000   
x2  0.000000  0.119805  0.0000  0.000000  0.000000  0.000000  0.000000   
x3  0.000000  0.000000  0.4667  0.000000  0.000000  0.000000  0.000000   
y1  0.

The *stats* module has various methods to calculate statistics and fit indices. However, there is a method *gather_statistics* that invokes them all:

In [10]:
from semopy.stats import gather_statistics

s = gather_statistics(opt_mlw)
print(s)

SEMStatistics(dof=35.0, ml=-333.86523324011876, fit_val=0.5083362656743606, chi2=(38.12521992557705, 0.3291802839852085), dof_baseline=55.0, chi2_baseline=730.6540868219104, rmsea=0.03473684815745261, cfi=0.9953745267193199, gfi=0.9478204247218975, agfi=0.26054087605889087, nfi=0.9478204247218975, tli=0.9927313991303599, aic=729.7304664802375, bic=801.5725979998631, params=[ParametersStatistics(value=1.4829393106346198, se=0.3991329374091464, zscore=3.7154019917791863, pvalue=0.0002028808933178805), ParametersStatistics(value=0.8373509275114408, se=0.09835312202791914, zscore=8.51371985196103, pvalue=0.0), ParametersStatistics(value=0.5723445128998603, se=0.22130946445228822, zscore=2.5861727798959597, pvalue=0.009704826891729246), ParametersStatistics(value=2.1803438171350416, se=0.13850284958372525, zscore=15.742230746068653, pvalue=0.0), ParametersStatistics(value=1.818498281423837, se=0.1519531025126112, zscore=11.967496887882973, pvalue=0.0), ParametersStatistics(value=1.256759361

A particular fit index/statistic can be invoked from *stats* module directly avoiding excess computations. For instance, let's say we want to calculate a GFI:

In [11]:
from semopy.stats import calc_gfi

print(calc_gfi(opt_gls))
print('MLW: {:.3f}, ULS: {:.3f}, GLS: {:.3f}, MLW after Adam after SLSQP: {:.3f}'.format(calc_gfi(opt_mlw),
                                                                                         calc_gfi(opt_uls),
                                                                                         calc_gfi(opt_gls),
                                                                                         calc_gfi(opt_mlw_adam)))

0.6844663290821422
MLW: 0.948, ULS: 0.997, GLS: 0.684, MLW after Adam after SLSQP: 0.948
